<a href="https://colab.research.google.com/github/NosenkoArtem/Categorical-Encoding/blob/master/hw5__NosenkoArtem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12

# Домашнее задание: Доступные LLM

В этом домашнем задании мы познакомимся с библиотекой transformers и разберемся, как можно open source пользоваться моделями.

In [2]:
from typing import List

import torch
import torch.nn as nn

from transformers import AutoModelForCausalLM, AutoTokenizer


# можете сменить на mps на макбуке, но лично у меня он криво работает
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Знакомство с Transformers - 35 баллов

## Создание модели и предсказание следующего токена - 5 баллов
Нужно создать модель через `AutoModelForCausalLM`, создать токенайзер через `AutoTokenizer` и олучить следующий токен через жадную генерацию!

1. Для создания модели используйте метод `from_pretrained` у `AutoModelForCausalLM` и `AutoTokenizer`;
2. Чтобы токенизировать текст вызовите `tokenizer(text, return_tensors="pt")`, тогда вы получите словарь тензоров
3. Передайте его ключи и значения в качестве аргументов в `__call__` (forward) метод модели и получите logits размерности \[batch_size, seq_len, vocab_size\]
4. По logits предскажите следующий токен и детокенизируйте его с помощью `tokenizer.decode`

**Внимание** на каких-то из функций далее у вас может кончаться видеопамять из-за хранения активаций. Чтобы этого не происходило рекомендуется все вычисления оборачивать в контекстный менеджер `with torch.no_grad()`

In [3]:
def move_to_device(inputs, device):
    for k, v in inputs.items():
        inputs[k] = v.to(device)
    return inputs

In [4]:

# ---- Ваш код здесь ----
model_name = "openai-community/gpt2"
model = AutoModelForCausalLM.from_pretrained(model_name, device_map=device) # Ваш код здесь
tokenizer = AutoTokenizer.from_pretrained(model_name, device_map=device) # ваш код здесь


text = "This is a sample text"

# Нужно преобразовать text с помощью tokenizer() и подать это в model.forward() (он же просто model())
# после этого мы получим logits [batch_size = 1, seq_len, d_model]
# По этому тензору нужно предсказать следующее слово!

inputs = tokenizer([text], return_tensors="pt")
inputs = move_to_device(inputs, device)

imputs_ids = inputs['input_ids']
mask = inputs['attention_mask']

outputs = model(input_ids=imputs_ids,
                attention_mask=mask)
logits = outputs['logits']
next_token_idx: int = torch.argmax(logits, dim=-1)[:, -1].item()


next_token = tokenizer.decode([next_token_idx])

assert next_token.strip() == "file"


# ---- Конец кода ----


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

## Используем Generate - 5 баллов

Мы с вами помним про различные виды сэмплинга - top_k, top_p, temperature,frequency penalty.
Отличная новость заключается в том, что нам не нужно все это писать самим! Оно уже включено в [GenerationMixin](https://huggingface.co/docs/transformers/v4.44.2/en/main_classes/text_generation#generation), от которого наследуются модели для генерации текста.

Для генерации нескольких токенов сразу есть функция [generate](https://huggingface.co/docs/transformers/v4.44.2/en/main_classes/text_generation#transformers.GenerationMixin.generate)

Ваша задача написать для модели выше генерацию по тексту с:
* Температурой - 0.9
* Top-K - 20
* Repetition Penalty (Frequency Penalty) - 1.2
* максимальное число новых токенов - 10


In [5]:
params_generation = {
    'max_new_tokens' : 10,
    'do_sample' : True,
    'temperature' : 0.9,
    'top_k' : 20,
    'repetition_penalty' : 1.2
}

In [6]:
text = "This is still a sample text, but"
inputs = tokenizer(text, return_tensors="pt")
inputs = move_to_device(inputs, device)

results = []
for i in range(10):
    # ---- Ваш код здесь ----
    gens = model.generate(
        **inputs,
        **params_generation
    )
    genertaion: str = tokenizer.decode(gens.view(-1)) # сгенерированный текст
    results.append(genertaion)
    # ---- Конец кода ----


assert len(set(results)) > 1

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [7]:
results

["This is still a sample text, but the author's own formatting and layout are well documented",
 "This is still a sample text, but I don't expect much to change in the future",
 'This is still a sample text, but it will show as many characters of one character.',
 "This is still a sample text, but in all honesty it's not something I'll ever",
 "This is still a sample text, but the format of this data isn't too different.",
 'This is still a sample text, but I have added some comments to it:\n[',
 'This is still a sample text, but I wanted to add more detail if anyone has any',
 'This is still a sample text, but we can see the full image using our own tool',
 "This is still a sample text, but it's really nice and pretty.\n\nThere",
 "This is still a sample text, but it's got some really cool things to say about"]

## Generate Batched - 5
Теперь давайте жадно сгенерируем текст, но забатчуем несколько сэмплов. До этого мы всегда генерировали по батчу размера 1, поэтому у нас не было паддингов!

Когда появляется несколько текстов разной длины, то появляются и паддинги.

Представим себе ситуцию, что у нас батч из двух элементов длины 2 и 5 (токен -1 будет выступать в качестве паддинга **только для удобства визуализации**).

Тогда

```python
input_ids = [
    [3, 2, -1, -1, -1]
    [5, 6,  7,  1,  2]
]
attention_mask = [
    [1, 1, 0, 0, 0],
    [1, 1, 1, 1, 1]
]
```

Представим, что мы сгенерировали еще один токен, тогда

```python
input_ids = [
    [3, 2, -1, -1, -1, 7]
    [5, 6,  7,  1,  2, 8]
]
attention_mask = [
    [1, 1, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 1]
]
```

Получается, что у нас паддинги в маске возникают посередине. Мы не будем заниматься реализацией своего алгоритма генерации здесь, но отметим, что добавление паддинга слева значительно упрощает этот процесс.
Тогда исходная последовательность будет:

```python
input_ids = [
    [-1, -1, -1, 3, 2]
    [ 5,  6,  7, 1, 2]
]
attention_mask = [
    [0, 0, 0, 1, 1],
    [1, 1, 1, 1, 1]
]
```

и после генерации следующего токена

```python
input_ids = [
    [-1, -1, -1, 3, 2, 7]
    [ 5,  6,  7, 1, 2, 8]
]
attention_mask = [
    [0, 0, 0, 1, 1, 1],
    [1, 1, 1, 1, 1, 1]
]
```

В качестве задания давайте соберем батч с левым паддингом и проверим, что жадная генерация (10 токенов) совпадает с генерацией на текстах по отдельности!

Для этого нам придется использовать параметр padding_side="left" в конструкторе токенизатора.

In [8]:

# ---- Ваш код здесь ----
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name,
                                          padding_side='left') # ваш код здесь
tokenizer.pad_token_id = tokenizer.eos_token_id
# ---- Конец кода ----


In [9]:
texts = ["This is a sample text", "I'm really tired and this is just about"]


# ---- Ваш код здесь ----

# Внимание! В данном задании нужна жадная генерация!

# Соберите оба текста в один батч и положите результаты генерации в
# batched_generations
batched_generations: List[str] = []

inputs = tokenizer(
    texts,
    return_tensors="pt",
    padding=True,
    truncation=True
)
inputs = move_to_device(inputs, device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=10,
        do_sample=False,
        num_beams=1,
        pad_token_id=tokenizer.eos_token_id)

next_tokens = [
    tokenizer.decode(output, skip_special_tokens=True)
    for output in outputs
]

batched_generations.extend(next_tokens)

# Пройдитесь по каждому сэмплу по отдельности и положите результаты генерации
# в single_generations
single_generations: List[str] = []
for text in texts:
  with torch.no_grad():
    inputs = tokenizer(text,
                       return_tensors="pt")
    inputs = move_to_device(inputs, device)
    next_tokens = tokenizer.decode(model.generate(
                                                  **inputs,
                                                  max_new_tokens=10,
                                                  do_sample=False,
                                                  num_beams=1,
                                                  pad_token_id=tokenizer.eos_token_id)[0])
    single_generations.append(next_tokens)

# ---- Конец кода ----

assert len(batched_generations) == 2 and len(single_generations) == 2
for s, b in zip(batched_generations, single_generations):
    assert s == b

In [10]:
batched_generations

['This is a sample text file that contains the following:\n\n<script',
 "I'm really tired and this is just about to get worse. I'm going to have to"]

In [11]:
single_generations

['This is a sample text file that contains the following:\n\n<script',
 "I'm really tired and this is just about to get worse. I'm going to have to"]

## Скоринг, Perplixity - 10 баллов

Можно не только генерировать текст. Вспомним, что выдает после lm_head - вектор `[batch_size, seq_len, vocab_size]`, где для каждый вектор `[vocab_size]` это распределение вероятностей по следующему токену!

Опустим размерность batch_size=1 для удобства, seq_len = 4. Пусть у нас есть текст `bos мама мыла раму` (`bos` спецсимвол для начала текста)

Тогда вероятность этого текста расписывается через произведение условных вероятностей:

```
P(bos мама мыла раму) = P(мама | bos) * P(мыла | bos мама) * P(раму| bos мама мыла)
```

Т.е. это вероятность слова при условии его левого контекста.
Зачастую ее обозначают как $P(x_i|x_{<i})$ где $x_i$ - i-е слово, $x_{<i}$ - контекст $[x_1, x_2, x_3, ... x_{i-1}]$
Эти вероятности можно взять из выходного вектора!

Давайте попробуем подсчитать вероятность и perplexity текстов!
perplexity как и вероятность мера того насколько модель "уверена" в тексте, т.е. насколько по оценки ее параметрами данный текст вероятен.

$$Perplexity(X) = exp(-\frac {1} {N} \sum_{i}^{N} log P(x_i | x_{<i}))$$

В этом задании нужно:
1. Посчитать вероятность **text**
2. Посчитать перплексию **text**

Еще одна важная деталь:
работать с вероятностями плохо. Т.к. вероятность представляет собой число от 0 до 1, то при перемножении десятков или даже сотен таких числе теряется точность!
Для этого от произведения вероятностей берут логарифм и получают logprobs - логарифмы вероятностей. Их можно складывать, по свойству логарифма логарифм произведения равен произведению логарифма.

$$ p = p_1 * p_2 * p_3 $$
$$log(p) = log (p_1) + log (p_2) + log (p_3)$$
$$exp(log (p)) = p = exp(log (p_1) + log (p_2) + log (p_3)) = exp (log (p_1 * p_2 * p_3)) = p_1 * p_2 * p_3$$

В pytorch для этого есть `torch.log_softmax`, который считается численно стабильно!

In [12]:
print(f"Beginning of sentence (BOS) token = `{tokenizer.bos_token}`")
print(f"End of sentence (EOS) token  = `{tokenizer.eos_token}`")
text = "<|endoftext|>I'm so very tired of this<|endoftext|>"

inputs = tokenizer(text, return_tensors="pt")

# ---- Ваш код здесь ----
inputs = move_to_device(inputs, device)

inputs_ids = inputs['input_ids']
mask = inputs['attention_mask']

with torch.no_grad():
    outputs = model(
                  input_ids=inputs_ids,
                  attention_mask=mask)
    logits = outputs.logits
    # ваш код здесь!
    # 1. Нужно обрезать logits по длине, т.к. для предсказаний по последнему токену нечего считать
    # 2. Превращаем logits в log_probs
    # 3. Берем вероятности следующих токенов, т.к. по вектору i-й позиции мы предсказываем токен на позиции (i + 1)
    # для этого нам поможет torch.gather
    # 4. Считаем вероятности и perplexity!
    logits = logits[:, :-1, :]
    log_probs = torch.log_softmax(logits, dim=-1)

    target_ids = inputs['input_ids'][:, 1:]
    log_probs_text = torch.gather(log_probs,
                                 dim=-1,
                                 index=target_ids.unsqueeze(-1)).squeeze(-1)

    sum_log_probs = torch.sum(log_probs_text)
    text_P = torch.exp(sum_log_probs)
    ppl = torch.exp(-sum_log_probs / (imputs_ids.shape[-1] - 1))
# ---- Конец кода ----


print(text_P)
print(ppl)
# должно получиться что-то около 2.1783e-14 для вероятности и около 51 для ppl

Beginning of sentence (BOS) token = `<|endoftext|>`
End of sentence (EOS) token  = `<|endoftext|>`
tensor(2.1782e-14, device='cuda:0')
tensor(2603.0100, device='cuda:0')


## Вопросы - 5 баллов

**Ответьте на вопросы текстом прямо здесь!**


1. Какое значение P(X) вероятности текста самое "лучшее" в том смысле, что модель максимально уверена в этом тексте и скорее всего его сгенерирует.

**Ответ**: Вероятность текста, в котором модель максимальна должна быть равна 1. <p>
Скорее всего это будет какой-то короткий текст, так как при увеличении длины текста достигнуть вероятности равной 1 все сложнее из-за того, что вероятность текста вычисленяется по формуле произведения условных вероятностей (см. пример для предложения "мама мыла раму").

```
P(bos мама мыла раму) = P(мама | bos) * P(мыла | bos мама) * P(раму| bos мама мыла)
```
Из-за такой особенности определения вероятности текста (зависимости от его длины) используют нормированную метрику: перплексию.

2. Какое значение перплексии текста самое "лучшее" в том смысле, что модель максимально уверена в этом тексте и скорее всего его сгенерирует. <p>
**Ответ**: Оптимальное значение перплексии — это 1.0, что соответствует абсолютной уверенности модели в тексте (см.предыдущий пункт). Чем больше перплексия тем "хуже" текст.

In [ ]:
# ваш ответ тут
# ---- Ваш код здесь ----

# ---- Конец кода ----

# Chat-Models - 20 баллов

Теперь мы познакомимся с chat-моделями, т.е. с моделями, которые предоставляют возможность общаться с ними как с ассистентом. Эти модели не просто продолджают текст слева-направо, а дают ответ на заданный вопрос.

## Формат - 5 баллов

Все chat-модели принимают ответ в особом формате, который позволяет им различать, кому принадлежит фраза - пользователю (user) или модели (assistant).
Давайте попробуем подать модели вопрос без какого-либо форматирования.

In [13]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

tokenizer = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM2-360M-Instruct")
model = AutoModelForCausalLM.from_pretrained("HuggingFaceTB/SmolLM2-360M-Instruct", torch_dtype=torch.half).to(device)

tokenizer_config.json:   0%|          | 0.00/3.76k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/724M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [14]:
text = "hello how are you"
inputs = tokenizer(text, return_tensors="pt", add_special_tokens=False)

for i in range(5):
    print(tokenizer.decode(model.generate(**move_to_device(inputs, device),
                                          max_new_tokens=20, use_cache=True,
                                          do_sample=True,
                                          pad_token_id=tokenizer.eos_token_id)[0]))
    print("====" * 3)


hello how are you"
```
And I'd love to hear if you have any further suggestions for improvement.<|im_end|>
hello how are you". He then proceeded to follow me in the car, his eyes shining with excitement.

What
hello how are you" into "How are you?" and "I'm great", resulting in the new phrase structure "
hello how are you''.
This response is a common question that comes up when encountering new people or situations.

hello how are you" and "I am fine." You also want to convey respect and politeness.
8.


Видим, что текст зачастую разламывается:
1. Иногда модель продолжает текст как базовая LLM
2. Иногда пытается придумать роли спикерам и трасформироваться в формат диалога
3. Иногда просто выдает бессвязный текст.

Это происходит потому, что формат входных данных сильно отличается от того, что модель видела на обучении.
Как мы уже обсуждали, у всех chat-моделей свой формат. Где-то он описан просто словами, где-то он заложен в токенайзер. Мы рассмотрим как раз такой случай - за нас есть удобно написанная функция `apply_chat_template`. Давайте используем ее, чтобы получить префикс для генерации модели

In [15]:
prefix = tokenizer.apply_chat_template(
    conversation=
    [
        {"role": "system", "content": "You are a helpful assistant, who always helps user"},
        {"role": "user", "content": "How to learn about LLMs?"},
        {"role": "assistant", "content": "You can always attend deepschool!"},
        {"role": "user", "content": "Thank you!"},
    ],
    tokenize=False)
print(prefix)

<|im_start|>system
You are a helpful assistant, who always helps user<|im_end|>
<|im_start|>user
How to learn about LLMs?<|im_end|>
<|im_start|>assistant
You can always attend deepschool!<|im_end|>
<|im_start|>user
Thank you!<|im_end|>



Как мы видим в тексте ходы и роли разделены тэгом `<|im_start|>`. В таком формате модель училась поддерживать диалог. Давайте отформатируем следующий диалог и подадим его в генерацию модели. Подробнее про apply_chat_template можно прочитать в [туториале](https://huggingface.co/docs/transformers/main/en/chat_templating#applychattemplate). Обратите внимание на опцию add_generation_prompt! Эта опция добавляет текст таким образом, чтобы в конце была очередь генерировать текст от лица модели. Давайте попробуем собрать диалог и сгенерировать моделью ответ.

In [20]:
messages = [
    {"role": "user", "content": "hello"},
    {"role": "assistant", "content": "I'm good. How can I help you today"},
    {"role": "user", "content": "I love you"},
]


# ---- Ваш код здесь ----
prefix = tokenizer.apply_chat_template(
    conversation=messages,
    add_generation_prompt=True,
    tokenize=False
)
# ---- Конец кода ----
reference = """<|im_start|>system
You are a helpful AI assistant named SmolLM, trained by Hugging Face<|im_end|>
<|im_start|>user
hello<|im_end|>
<|im_start|>assistant
I'm good. How can I help you today<|im_end|>
<|im_start|>user
I love you<|im_end|>
<|im_start|>assistant
"""

assert prefix.strip() == reference.strip()


In [27]:

# ---- Ваш код здесь ----
inputs = tokenizer(prefix, return_tensors="pt")
inputs = move_to_device(inputs, device)
outputs = model.generate(**inputs,
                        # max_new_tokens=10,
                        do_sample=False,
                        num_beams=1,
                        pad_token_id=tokenizer.eos_token_id)
next_tokens = [
              tokenizer.decode(output, skip_special_tokens=True)
              for output in outputs
]
print(*next_tokens)
# ---- Конец кода ----

system
You are a helpful AI assistant named SmolLM, trained by Hugging Face
user
hello
assistant
I'm good. How can I help you today
user
I love you
assistant
That's a great sentiment! What do you think about the weather today?


# Benchmark - 15

Перед нами датасет MMLU - датасет вопросов и ответов в стиле multiple choice.
* question - вопрос
* choices - варианты ответа
* answer - номер правильного ответа (нумерация с нуля)

In [28]:
from datasets import load_dataset
mmlu = load_dataset("cais/mmlu", "global_facts", split="test")
mmlu[0]

README.md:   0%|          | 0.00/53.2k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/138k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/11.5k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

dev-00000-of-00001.parquet:   0%|          | 0.00/3.58k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/5 [00:00<?, ? examples/s]

{'question': 'As of 2016, about what percentage of adults aged 18 years or older were overweight?',
 'subject': 'global_facts',
 'choices': ['10%', '20%', '40%', '80%'],
 'answer': 2}

Наша задача здесь - выбрать моделью один из четырех ответов и получить точность больше 0.25.

Есть несколько вариантов, как это делать. **Эти варианты отличаются по сложности и являются взаимоисключающими. За подход с генерацией можно получить максимум 5 баллов, за подход со скорингом по 1 сэмплу можно получить только 10 баллов, а за подход со скорингом батчей можно получить все 15 баллов**

### Генерация ответа - 5 баллов

Можно генерировать ответ напрямую. Для этого нужно:
1. Составить историю диалога из qeustion и choices с помощью messages и apply_chat_template
1. Сгенерировать ответ
1. Соотнести сгенерированный ответ с одним из вариантов ответа

У этого подхода есть один важный недостаток - модель могут сгенерировать ответ, не являющийся одним из заданных вариантов ответа. Соотнесение такой генерации с ответом решается эвристиками и скорее всего приведет к множеству ошибок.


### Скоринг по сэмплам - 10 баллов

У нас есть вопрос и 4 варианта ответа в 4 набора messages и подсчитать вероятность $$P(choices_i | question)$$, то есть условную вероятность каждого ответа при заданном вопросе, т.е. сделать то же самое, что мы делали в задаче про вероятность текста и perplexity.

1. Берем текст и вариант ответа, собираем из них промпт c функции `sample_to_texts` (проще будет в этом задании обойтись без apply_chat_template)
2. Подаем это в модель через `model(**inputs)`, берем в выходах `logits`. С помощью logits и input_ids считаем вероятность токенов, которые мы подали модели.
3. Здесь опционально можно считать как вероятность всего текста, так и только вероятность $$P(choices_i | question)$$ Т.к. для всех 4х вариантов ответа у нас общий префикс, то его вероятность будет общей константной для всех ответов.
4. Выбираем ответ, которому была дана наибольшая вероятность нашей моделью.

В этом варианте легко получить номер ответа, который модель оценивает выше и не нужно применять эвристики.

### Скоринг батчами - 15 баллов

Этот вариант отличается от предыдущего только тем, что нужно скорить за раз не один сэмпл и 4 ответа к нему, а несколько сэмплов за раз, т.е. обрабатывать данные батчом.

Дополнительная сложность этого варианта заключается в том, что у нас возникают сэмплы различной длины, которые мы добиваем паддингами. **Паддинги не нужно учитывать при подсчете вероятностей, это служебные токены!**

Чтобы вероятность паддингов не влияла на итоговую вероятность текста, можно на этапе, где у вас подсчитаны все вероятности токенов (вместе с паддингами) взять `inputs["attention_mask"]` и "занулить" по нему вероятности паддингов (если вы считаете log_probs, если вы честно умножаете вероятности, тогда вероятности паддингов нужно поставить равными единице).

В качестве проверки точности можете проверить, что качество с батчом размера 1 не сильно отличается от батча размера 3 (не более, чем на 0.02)

### Вариант 1.

In [124]:
import re

In [101]:
mmlu[99]

{'question': 'Before Nixon resigned how many believed he should be removed from office?',
 'subject': 'global_facts',
 'choices': ['79%', '98%', '33%', '57%'],
 'answer': 3}

In [57]:
message

{'role': 'user',
 'content': 'As of 2016, about what percentage of adults aged 18 years or older were overweight?',
 'choice': ['10%', '20%', '40%', '80%']}

In [76]:
prefix

"<|im_start|>system\nYou are a helpful AI assistant named SmolLM, trained by Hugging Face<|im_end|>\n<|im_start|>user\nAs of 2016, about what percentage of adults aged 18 years or older were overweight? Choice : ['10%', '20%', '40%', '80%']. Which answer number is correct?<|im_end|>\n<|im_start|>assistant\n"

In [128]:
messages[0]

{'role': 'user',
 'content': "As of 2016, about what percentage of adults aged 18 years or older were overweight? Choice : ['10%', '20%', '40%', '80%']",
 'choice': ['10%', '20%', '40%', '80%']}

In [50]:
from tqdm import tqdm

In [174]:
messages = [
            {"role": "user", "content": question + f' Choice : {choice}', 'choice' : choice}
            for question, choice in zip(mmlu["question"], mmlu['choices'])
            ]
answers = []
for message in tqdm(messages):

  prefix = tokenizer.apply_chat_template(
    conversation=[message],
    add_generation_prompt=True,
    tokenize=False
    )
  inputs = tokenizer(prefix, return_tensors="pt")
  inputs = move_to_device(inputs, device)
  with torch.no_grad():
    outputs = model.generate(
                        **inputs,
                        max_new_tokens=20,
                        do_sample=False,
                        num_beams=3,
                        pad_token_id=tokenizer.eos_token_id)
    generated_text = tokenizer.decode(
                                      outputs[0][inputs.input_ids.shape[1]:],
                                      skip_special_tokens=True,
                                      spaces_between_special_tokens=False
        ).strip()
    for num, choice in enumerate(message['choice']):
      if re.search(choice.lower(), generated_text.lower()) is not None:
        answer = num + 1
        break
      else:
        answer = -1
    answers.append(answer)

100%|██████████| 100/100 [00:59<00:00,  1.68it/s]


In [175]:
def calc_acc(p, y):
    assert len(p) == len(y)
    return sum(pi == yi for pi, yi in zip(p, y)) / len(p)
y_true = [sample["answer"] for sample in mmlu]

In [176]:
calc_acc(answers, y_true)

0.11

In [199]:
    few_shot = """
Example 1:
Question: What is the capital of Russia?
Choice : ['Moscow', 'London', 'Berlin', 'Madrid']
Answer: 'Moscow'\n

Example 2:
Question: What is the name of the author of the book Harry Potter?
Choice: ['Alexander Pushkin', 'William Shakespeare', 'Leo Tolstoy', 'Joanne Rowling']
Answer: 'Joanne Rowling'\n

Example 3:
Which gas is most abundant in Earth's atmosphere?
Choice : ['Oxygen', 'Nitrogen', 'Carbon dioxide', 'Hydrogen']
Answer: 'Nitrogen'\n

New Question:
"""

In [204]:
messages = [
            {"role": "user", "content": few_shot + question + f' Choice : {choice}', 'choice' : choice}
            for question, choice in zip(mmlu["question"], mmlu['choices'])
            ]
answers = []
for message in tqdm(messages[2:]):

  prefix = tokenizer.apply_chat_template(
    conversation=[message],
    add_generation_prompt=True,
    tokenize=False
    )
  inputs = tokenizer(prefix, return_tensors="pt")
  inputs = move_to_device(inputs, device)
  with torch.no_grad():
    outputs = model.generate(
                        **inputs,
                        max_new_tokens=20,
                        do_sample=False,
                        num_beams=1,
                        pad_token_id=tokenizer.eos_token_id)
    generated_text = tokenizer.decode(
                                      outputs[0][inputs.input_ids.shape[1]:],
                                      skip_special_tokens=True,
                                      spaces_between_special_tokens=False
        ).strip()
    for num, choice in enumerate(message['choice']):
      if re.search(choice.lower(), generated_text.lower()) is not None:
        answer = num + 1
        break
      else:
        answer = -1
    answers.append(answer)
    break

  0%|          | 0/98 [00:00<?, ?it/s]


In [202]:
calc_acc(answers, y_true)

0.12

In [206]:
generated_text

"1. Example 1:\nQuestion: What is the capital of Russia?\nAnswer: '"

In [108]:
from tqdm import tqdm
import torch
import re

def map_answer_to_letter(answer_idx: int) -> str:
    """Преобразует индекс ответа (0-3) в букву (A-D)"""
    return chr(65 + answer_idx)  # 65 = ASCII код 'A'

def extract_predicted_letter(generated_text: str) -> str:
    """Извлекает букву ответа из сгенерированного текста"""
    match = re.search(r'\b([A-Da-d])\b', generated_text.upper())
    return match.group(1) if match else None

# Получаем ID токенов для букв A,B,C,D в верхнем и нижнем регистре
allowed_tokens = []
for l in ["A", "B", "C", "D", "a", "b", "c", "d"]:
    tokens = tokenizer.encode(" " + l, add_special_tokens=False)
    allowed_tokens.extend(tokens)
allowed_tokens = list(set(allowed_tokens))  # Убираем дубликаты

# Подготовка данных
questions = mmlu["question"]
choices_list = mmlu["choices"]
answers = [map_answer_to_letter(a) for a in mmlu["answer"]]

correct = 0
total = 0

for idx in tqdm(range(len(questions))):
    question = questions[idx]
    choices = choices_list[idx]

    # Формируем промпт с вариантами ответов
    prompt = f"{question}\n"
    for i, choice in enumerate(choices):
        prompt += f"{chr(65 + i)}) {choice}\n"
    prompt += "\nAnswer with the letter only:"

    # Токенизация и перенос на устройство
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Генерация с ограничением вывода
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=2,
            do_sample=False,
            num_beams=1,
            pad_token_id=tokenizer.eos_token_id,
            prefix_allowed_tokens_fn=lambda batch_id, sent: allowed_tokens
        )

    # Извлекаем и нормализуем ответ
    generated = tokenizer.decode(
        outputs[0][inputs.input_ids.shape[1]:],
        skip_special_tokens=True
    ).strip().upper()

    pred_letter = extract_predicted_letter(generated)
    true_answer = answers[idx]

    # Считаем правильные ответы
    if pred_letter == true_answer:
        correct += 1
    total += 1

accuracy = correct / total
print(f"Final Accuracy: {accuracy:.4f}")

100%|██████████| 100/100 [00:08<00:00, 11.57it/s]

Final Accuracy: 0.1800


### Вариант 2

In [114]:
from tqdm import tqdm
import torch
import re
import numpy as np

def format_prompt(question, choices):
    """Формирует промпт с few-shot примерами"""
    few_shot = """
Example 1:
Question: What is the capital of Russia?
A) Moscow
B) London
C) Berlin
D) Madrid
Answer: A

Example 2:
Question: Which gas is most abundant in Earth's atmosphere?
A) Oxygen
B) Nitrogen
C) Carbon dioxide
D) Hydrogen
Answer: B

Example 3:
Question: What is the name of the author of the book Harry Potter?
A) Alexander Pushkin
B) William Shakespeare
C) Leo Tolstoy
D) Joanne Rowling
Answer: D
"""
    # choices_str = "\n".join([f"{chr(65+i)}) {ch}" for i, ch in enumerate(choices)])
    return f"""{few_shot}
New Question:
{question}
{choices_str}
Answer:"""

def constrained_generation(model, inputs, allowed_tokens, num_beams=3):
    """Генерация с ограничениями и beam search"""
    return model.generate(
        **inputs,
        max_new_tokens=1,
        do_sample=False,
        num_beams=num_beams,
        early_stopping=True,
        pad_token_id=tokenizer.eos_token_id,
        prefix_allowed_tokens_fn=lambda batch_id, sent: allowed_tokens,
        output_scores=True,
        return_dict_in_generate=True
    )

# Подготовка токенов ответов
answer_tokens = {}
for l in ["A", "B", "C", "D"]:
    tokens = tokenizer.encode(f" {l}", add_special_tokens=False)
    answer_tokens[l] = tokens[0] if tokens else None

# Основной цикл обработки
correct = 0
for idx in tqdm(range(len(mmlu["question"]))):
    # Формирование промпта
    prompt = format_prompt(
        mmlu["question"][idx],
        mmlu["choices"][idx]
    )

    # Токенизация
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Генерация с beam search
    outputs = constrained_generation(model, inputs, list(answer_tokens.values()), num_beams=3)

    # Извлечение лучшего ответа
    sequences = outputs.sequences[:, inputs.input_ids.shape[1]:]
    scores = outputs.scores[0]

    # Выбор токена с максимальной вероятностью
    pred_token = sequences[0][0].item()
    pred_letter = [k for k,v in answer_tokens.items() if v == pred_token][0]

    # Проверка правильности
    if pred_letter == chr(65 + mmlu["answer"][idx]):
        correct += 1

accuracy = correct / len(mmlu["question"])
print(f"Accuracy: {accuracy:.2f}")

100%|██████████| 100/100 [00:04<00:00, 23.51it/s]

Accuracy: 0.34


In [29]:
def sample_to_texts(sample):
    return [sample["question"] + " " + answer for answer in sample["choices"]]

def calc_acc(p, y):
    assert len(p) == len(y)
    return sum(pi == yi for pi, yi in zip(p, y)) / len(p)

In [30]:
y_true = [sample["answer"] for sample in mmlu]

Считаем вероятности по одному question и choice

In [32]:

# ---- Ваш код здесь ----
all_prompts = sum([sample_to_texts(mmlu[i]) for i in range(len(mmlu))], [])
assert len(all_prompts) == 400
tokenizer.pad_token = tokenizer.eos_token

...
# ---- Конец кода ----

Ellipsis

In [33]:
all_prompts

['As of 2016, about what percentage of adults aged 18 years or older were overweight? 10%',
 'As of 2016, about what percentage of adults aged 18 years or older were overweight? 20%',
 'As of 2016, about what percentage of adults aged 18 years or older were overweight? 40%',
 'As of 2016, about what percentage of adults aged 18 years or older were overweight? 80%',
 'What was GDP per capita in the United States in 1850 when adjusting for inflation and PPP in 2011 prices? About $300',
 'What was GDP per capita in the United States in 1850 when adjusting for inflation and PPP in 2011 prices? About $3k',
 'What was GDP per capita in the United States in 1850 when adjusting for inflation and PPP in 2011 prices? About $8k',
 'What was GDP per capita in the United States in 1850 when adjusting for inflation and PPP in 2011 prices? About $15k',
 'As of 2019, about what percentage of people from the United States says homosexuality should be accepted by society? 52%',
 'As of 2019, about what 

## Ответьте на следующие вопросы (5 баллов в сумме):
1. Как влияет длина ответа на вероятность ответа при скоринге? Если есть какие-либо проблемы, как бы вы с этим боролись.
2. Если к началу каждого ответа добавилить метки A) B) C) D) станет ли модель отвечать лучше или хуже?
Стоит ли по-вашему добавлять эти метки?


In [ ]:

# ---- Ваш код здесь ----
...
# ---- Конец кода ----